# 星际争霸II数学模型 - 分析演示

这个notebook展示如何使用本项目进行单位分析。

In [ ]:
# 导入必要的库
import sys
from pathlib import Path
sys.path.append(str(Path.cwd().parent))

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# 设置中文字体
plt.rcParams['font.sans-serif'] = ['Arial Unicode MS', 'SimHei']
plt.rcParams['axes.unicode_minus'] = False

# 导入项目模块
from src.core.cev_calculator import CEVCalculator, UnitParameters
from src.data.csv_data_loader import CSVDataLoader

## 1. 加载数据

In [ ]:
# 加载单位数据
loader = CSVDataLoader()
units_data = loader.load_units_data()

print(f"成功加载 {len(units_data)} 个单位")

# 转换为DataFrame查看
df = pd.DataFrame(units_data)
df.head()

## 2. 单位CEV计算

In [ ]:
# 初始化计算器
calculator = CEVCalculator()

# 选择一个单位进行详细分析
unit_name = "陆战队员"
commander = "诺娃·泰拉"

# 找到单位数据
unit_data = next((u for u in units_data 
                 if u['name'] == unit_name and u['commander'] == commander), None)

if unit_data:
    # 创建单位参数
    unit = UnitParameters(
        name=unit_data['name'],
        commander=unit_data['commander'],
        mineral_cost=unit_data['mineral_cost'],
        gas_cost=unit_data['gas_cost'],
        supply_cost=unit_data['supply_cost'],
        base_dps=unit_data['base_dps'],
        hp=unit_data['hp'],
        armor=unit_data['armor'],
        shields=unit_data['shields'],
        range=unit_data['range'],
        speed=unit_data['speed'],
        abilities=unit_data['abilities']
    )
    
    # 计算CEV
    result = calculator.calculate_cev(unit, time_seconds=600)
    
    print(f"{unit_name} ({commander}) 中期游戏评估:")
    print(f"CEV: {result['cev']:.2f}")
    print(f"有效成本: {result['effective_cost']:.2f}")
    print(f"有效DPS: {result['effective_dps']:.2f}")
    print(f"有效HP: {result['effective_hp']:.2f}")
    print(f"λ(t): {result['lambda']:.3f}")

## 3. 批量评估和排名

In [ ]:
# 评估所有单位
results = []

for unit_data in units_data:
    # 跳过免费单位
    if unit_data['equivalent_cost'] == 0:
        continue
    
    # 创建单位参数
    unit = UnitParameters(
        name=unit_data['name'],
        commander=unit_data['commander'],
        mineral_cost=unit_data['mineral_cost'],
        gas_cost=unit_data['gas_cost'],
        supply_cost=unit_data['supply_cost'],
        base_dps=unit_data['base_dps'],
        hp=unit_data['hp'],
        armor=unit_data['armor'],
        shields=unit_data['shields'],
        range=unit_data['range'],
        speed=unit_data['speed'],
        abilities=unit_data['abilities']
    )
    
    # 计算中期CEV
    cev_result = calculator.calculate_cev(unit, time_seconds=600)
    
    results.append({
        'commander': unit_data['commander'],
        'unit_name': unit_data['name'],
        'cev': cev_result['cev'],
        'effective_cost': cev_result['effective_cost'],
        'cost_efficiency': cev_result['cev'] / cev_result['effective_cost'] if cev_result['effective_cost'] > 0 else 0
    })

# 转换为DataFrame
df_results = pd.DataFrame(results)

# 显示Top 10
print("Top 10 单位 (按CEV排序):")
df_results.nlargest(10, 'cev')[['unit_name', 'commander', 'cev', 'cost_efficiency']]

## 4. 数据可视化

In [ ]:
# 1. CEV分布直方图
plt.figure(figsize=(10, 6))
plt.hist(df_results['cev'], bins=30, edgecolor='black', alpha=0.7)
plt.xlabel('CEV值')
plt.ylabel('单位数量')
plt.title('单位CEV分布')
plt.grid(True, alpha=0.3)
plt.show()

# 2. 指挥官平均CEV
commander_avg = df_results.groupby('commander')['cev'].mean().sort_values(ascending=False)

plt.figure(figsize=(12, 8))
commander_avg.head(10).plot(kind='bar')
plt.xlabel('指挥官')
plt.ylabel('平均CEV')
plt.title('各指挥官平均单位CEV (Top 10)')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()

## 5. 成本效率分析

In [ ]:
# 成本-效能散点图
plt.figure(figsize=(12, 8))

# 按指挥官着色
commanders = df_results['commander'].unique()[:6]  # 选择6个指挥官
colors = plt.cm.tab10(range(len(commanders)))

for i, cmd in enumerate(commanders):
    cmd_data = df_results[df_results['commander'] == cmd]
    plt.scatter(cmd_data['effective_cost'], cmd_data['cev'], 
               c=[colors[i]], label=cmd, alpha=0.6, s=80)

plt.xlabel('有效成本')
plt.ylabel('战斗效能值 (CEV)')
plt.title('单位成本-效能分析')
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

# 性价比Top 10
print("\n性价比最高的10个单位:")
df_results.nlargest(10, 'cost_efficiency')[['unit_name', 'commander', 'cost_efficiency', 'cev']]

## 6. 时间演化分析

In [ ]:
# 分析λ(t)函数
import numpy as np

time_points = np.linspace(0, 1800, 100)
lambda_values = [calculator.calculate_lambda(t) for t in time_points]

plt.figure(figsize=(10, 6))
plt.plot(time_points/60, lambda_values, 'b-', linewidth=2)
plt.xlabel('游戏时间 (分钟)')
plt.ylabel('λ(t)')
plt.title('人口压力因子λ(t)随时间变化')
plt.grid(True, alpha=0.3)
plt.axvline(x=3, color='r', linestyle='--', alpha=0.5, label='早期(3分钟)')
plt.axvline(x=10, color='g', linestyle='--', alpha=0.5, label='中期(10分钟)')
plt.axvline(x=20, color='purple', linestyle='--', alpha=0.5, label='后期(20分钟)')
plt.legend()
plt.show()